In this project we are trying to price an Equity protection certificate, Intesa Sanpaolo MAX LONG CAP CERTIFICATES on MIB ESG DECREMENT 5% (EUR - NR) Index due 31.12.2029 (ISIN code:XS2556920119), using Monte Carlo simulation.
But what is an Equity protection certificate? In short gives us the chance to participate to the Long of a certain underlying without incurring in a loss in case markets moves versus us. Gives a protection based on the contract and has a cap level making it similiar to a barrier option.
In order to price it we used the following enhancements:
-Vasicek model for the evolution of interest rates
-Heston model for the evolution of the underlying considering also a stochastic volatility
-The computation of the Greeks using numerical approximation which: Delta, Gamma, Vega, Rho and Theta
-The antithetic variance reduction
-The issuer credit risk using the hazard rates

In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

Here we can see a part of the details of our certificate and some parameters used in Vasicek and Heston Model.
Parameters used for short rates and heston are the main cause in change of price of our certificate that can make the price differ even in high amounts from the one priced on the market, but why?
The reasons are mainly based on different paramaters used in our models that reflect different market conditions(if no calibration involved):
    -The discount factors impacts the price of certificate (inverted yield curve or normal curves, etc.), so the parameters in Vasicek gives us different market realities causing different prices for each market reality
    -We can find ourselves in Risk-ON and Risk-OFF markets, highlighted by the presence of high/low volatilities where investors move within different market products causing the rise/fall of different products. Heston model helps us to evolve volatility in time which applied to our underlying gives good estimates of how index moves
Since we are using montecarlo simulations, we would need to discretize our formulas and the choice would be between Miler or Euler discretization, in our case we used the Euler discretization (we can see how to do it here: https://frouah.com/finance%20notes/Euler%20and%20Milstein%20Discretization.pdf)


In [3]:
#Certificate details
S0 = 763.91  #This is our Initial Reference Value for the underlying
issue_price = 1000.0 #This is the nominal of the Equity protection certificate
maturity = 7 #Certificate life time

#Vasicek parameters
r0 = 0.02 #Initial interest spot rate
alpha = 0.2 #This is the speed of mean reversion
gamma = 0.03 #Long term mean interest rate
r_vol = .07  #The volatility of interest rates

#Heston parameters
kappa = .2 #Mean reversion speed
theta = 0.09 #Long-term mean variance under risk-neutral dynamics
v0 = 0.4   #Initial squared volatility
sigma = 0.5  #VolofVol

Assumptions we used in order to price:
    -Why Vasicek?
        Unlike in the past where negative rates were seen by academic/economic figures in bad way and had low rate presence, today we see a different reality where negative rates can happen and can stay so for long times.
    -Why Heston?
        Heston has some properties that are useful to get a good price estimate, such as: Correlation dynamics between price/volatility, can capture volatility smile thanks to the stochastic volatility presence and is not very difficult to implement with MC.
The formulas used for this certificate can be seen in Final Terms documents: https://www.intesasanpaolo.prodottiequotazioni.com/EN/Download/Asset?id=2a3f7a80-aa22-470d-a662-0626f6489359.


In [6]:
def MCS(S0, issue_price, r0, alpha, gamma, r_vol, kappa, theta, v0, maturity, sigma, n_simulations, n_paths):
    cap_percentage = 1
    cap_level = S0 * cap_percentage #it will be equal to IRV
    multiplier = issue_price / S0
    participation_factor = 1
    initial_percentage = 1
    T = maturity #years
    premium_percentage = 0.0393
    premium_gearing = 2
    M = n_simulations
    N = n_paths
    np.random.seed(232)   #This is our seed used to generate the random normal numbers
    dt = 1 / N
    corr_matrix = np.array([[1.0, 0.5, 0.2],  #Here we have the correlation matrix, starting from first row we have: r/r, r/s and r/v
                            [0.5, 1.0, 0.3],
                            [0.2, 0.3, 1.0]])
    cholesky_factor = np.linalg.cholesky(corr_matrix) #Cholesky decomposition gives us as a result a lower triangular matrix that we use for the distribution correlations
    total_payoffs = [] #Qui creamo un vettore di payoffs per rendere più facile il calcolo della varianza/media
    years = np.arange(1, T+1, 1)
    mean = 0

    for i in range(M):
        r = r0
        S = S0
        V = v0
        sum_r = 0
        payoff = 0
        discounted_payoffs = 0
        discount_issue_price = 0
        for y in years:
            for s in range(N):
                Z_r = np.random.normal(0.0, 1.0) #Our random normal, I put in parenthesis mean 0.0 and variance 1.0 just to be sure he gives me this output
                Z_s = np.random.normal(0.0, 1.0)
                Z_v = np.random.normal(0.0, 1.0)
                eta_r = Z_r * cholesky_factor[0, 0]
                eta_s = Z_r * cholesky_factor[1, 0] + Z_s * cholesky_factor[1, 1]
                eta_v = Z_r * cholesky_factor[2, 0] + Z_s * cholesky_factor[2, 1] + Z_v * cholesky_factor[2, 2]

                #Stochastic Vasicek
                r = r + alpha * (gamma - r) * dt + r_vol * np.sqrt(dt) * eta_r
                sum_r += r

                #Heston underlying
                S = S * math.exp((r-0.5*V)*dt + np.sqrt(V) * np.sqrt(dt)* eta_s)

                # Apply stochastic volatility based on Heston model
                V = V + kappa*(theta-V)*dt + sigma * np.sqrt(V)*np.sqrt(dt) * eta_v
                if V<0:
                    V=abs(V) #We need to avoid negative V, and we use a reflection here that gives us absolute value in case of negative V

            #Here we have the calculation of our yearly coupons (called premium remuneration) that pays us based on underlying performance
            if y in years:
                participation_remuneration = issue_price * np.maximum(0, premium_percentage * (1 + premium_gearing * ((S - S0)/S0)))
                payoff += participation_remuneration
            #This is the final amount that the investor receveis at maturity
            if y == maturity:
                settlement_amount = np.minimum(cap_level, np.maximum(initial_percentage * S0, (S0 + 1 * (S - S0) * multiplier))) * 1
                payoff += settlement_amount
            #Trapezoidal rule for the short rates that we use after to get the discount factors and discount our payoffs (coupon + settlement amount)
            integral_r=(0.5*(r0-r) + sum_r) * dt
            discount_factor = np.exp(-integral_r)
            discounted_payoffs = payoff * discount_factor

        total_payoffs.append(discounted_payoffs)

    variance = np.std(total_payoffs)
    mean = np.mean(total_payoffs)
    MC_error = np.sqrt(variance/M)
    mean, variance, MC_error

    #print("Price", mean)
    #print("Standard error", std_error)
    return mean

The price we get has a difference of 2€ from the one in the market which is a very good result.
Why our price is so near the real price?
    Means that the parameters we chose reflects exactly or near the market conditions we are in. Obviously for better results we would like to calibrate our model.

In [7]:
#Here we can recall our function and put the different paramaters for our models and certificate details in order to price the certificate
MCS(763.91, 1000, .02, 0.2, .03, .07, .2, 0.09, 0.4, 7, .5, 1000, 250)

1015.6580269116268

Here we have our issuer credit risk. In order to compute and make this function I followed the idea of Hazard Rates presented in the book "Options, Futures and other Derivates" by Hull on page 563.
Since Intesa san paolo credit rating is BBB, years and avg_hazard_rates were extracted from table 24.1 and used in the function in order to discount the price given my our MC simulation.

In [8]:
def issuer_credit_risk(price, maturity):
    years = [1, 2, 3, 4, 5, 7, 10, 15]
    avg_hazard_rates = [0.0016, 0.0045, 0.0078, 0.0117, 0.0158, 0.0233, 0.0332, 0.0469]
    count = 0
    if maturity not in years:
        raise ValueError("Invalid maturity. Available options are: " + ", ".join(str(year) for year in years))
    else:
        for i in years:
            if i == maturity:
                count += 1
            default_probability = 1 - math.exp(-avg_hazard_rates[count]*maturity)
        adjusted_price = price * (1 - default_probability)

    return adjusted_price


In [9]:
price = MCS(763.91, 1000, .02, 0.2, .03, .07, .2, 0.09, 0.4, 7, .5, 1000, 250)
issuer_credit_risk(price, 7) #Price adjusted by the Hazard rates


984.1634429270154

In [16]:
def MCS_antithetic(S0, issue_price, r0, alpha, gamma, r_vol, kappa, theta, v0, maturity, sigma, n_simulations, n_paths):
    cap_percentage = 1
    cap_level = S0 * cap_percentage #it will be equal to IRV
    multiplier = issue_price / S0
    participation_factor = 1
    initial_percentage = 1
    T = maturity #years
    premium_percentage = 0.0393
    premium_gearing = 2
    M = n_simulations
    N = n_paths
    np.random.seed(232)
    dt = 1 / N
    corr_matrix = np.array([[1.0, 0.5, 0.2],
                            [0.5, 1.0, 0.3],
                            [0.2, 0.3, 1.0]])
    cholesky_factor = np.linalg.cholesky(corr_matrix)
    total_payoffs_anti = []
    total_payoffs = [] #Qui creamo un vettore di payoffs per rendere più facile il calcolo della varianza media
    years = np.arange(1, T+1, 1)
    mean = 0

    for i in range(M):
        r_anti = r0
        S_anti = S0
        V_anti = v0
        sum_r_anti = 0
        payoff_anti = 0
        discounted_payoffs_anti = 0
        discount_issue_price_anti = 0
        r = r0
        S = S0
        V = v0
        sum_r = 0
        payoff = 0
        discounted_payoffs = 0
        discount_issue_price = 0
        for y in years:
            for s in range(N):
                Z_r = np.random.normal(0.0, 1.0)
                Z_s = np.random.normal(0.0, 1.0)
                Z_v = np.random.normal(0.0, 1.0)
                eta_r = Z_r * cholesky_factor[0, 0]
                eta_s = Z_r * cholesky_factor[1, 0] + Z_s * cholesky_factor[1, 1]
                eta_v = Z_r * cholesky_factor[2, 0] + Z_s * cholesky_factor[2, 1] + Z_v * cholesky_factor[2, 2]

                #antithetic variables
                Z_r_anti = -(np.random.normal(0.0, 1.0))
                Z_s_anti = -(np.random.normal(0.0, 1.0))
                Z_v_anti = -(np.random.normal(0.0, 1.0))
                eta_r_anti = Z_r_anti * cholesky_factor[0, 0]
                eta_s_anti = Z_r_anti * cholesky_factor[1, 0] + Z_s_anti * cholesky_factor[1, 1]
                eta_v_anti = Z_r_anti * cholesky_factor[2, 0] + Z_s_anti * cholesky_factor[2, 1] + Z_v_anti * cholesky_factor[2, 2]


                r = r + alpha * (gamma - r) * dt + r_vol * np.sqrt(dt) * eta_r
                sum_r = sum_r + r

                r_anti = r_anti + alpha * (gamma - r_anti) * dt + r_vol * np.sqrt(dt) * eta_r_anti
                sum_r_anti = sum_r_anti + r_anti

                # Apply stochastic volatility based on Heston model
                V = V + kappa*(theta-V)*dt + sigma *np.sqrt(V)*np.sqrt(dt)*eta_v
                if V < 0:
                    V = abs(V)

                V_anti = V_anti + kappa*(theta-V_anti)*dt + sigma *np.sqrt(V_anti)*np.sqrt(dt)*eta_v_anti
                if V_anti < 0:
                    V_anti = abs(V_anti)

                #Heston underlying
                S = S * math.exp((r-0.5*V)*dt + np.sqrt(V) * np.sqrt(dt)* eta_s)

                S_anti = S_anti * math.exp((r_anti - 0.5*V_anti)*dt + np.sqrt(V_anti)*np.sqrt(dt)*eta_s_anti)

            if y in years:
                participation_remuneration = issue_price * np.maximum(0, premium_percentage * (1 + premium_gearing * ((S - S0)/S0)))

                payoff += participation_remuneration

                participation_remuneration_anti = issue_price * np.maximum(0, premium_percentage * (1 + premium_gearing * (S_anti - S0) / S0))

                payoff_anti += participation_remuneration_anti

            if y == maturity:
                settlement_amount = np.minimum(cap_level, np.maximum(initial_percentage * S0, (S0 + 1 * (S - S0) * multiplier))) * 1

                payoff += settlement_amount

                settlement_amount_anti = np.minimum(cap_level, np.maximum(initial_percentage * S0, (S0 + 1 * (S_anti - S0) * multiplier))) * 1

                payoff_anti += settlement_amount_anti


            integral_r=(0.5*(r0-r) + sum_r) * dt
            integral_r_anti = (0.5*(r0-r_anti) + sum_r_anti) * dt

            discount_factor = np.exp(-integral_r)
            discount_factor_anti = np.exp(-integral_r_anti)

            discounted_payoffs = payoff * discount_factor
            discounted_payoffs_anti = payoff_anti * discount_factor_anti

        total_payoffs.append(discounted_payoffs)
        total_payoffs_anti.append((discounted_payoffs_anti+discounted_payoffs)/2)

    variance = np.std(total_payoffs)
    variance_anti = np.std(total_payoffs_anti)

    mean = np.mean(total_payoffs)
    mean_anti = np.mean(total_payoffs_anti)

    MC_error = np.sqrt(variance/M)
    MC_error_anti = np.sqrt(variance_anti/M)



    print("Price with no variance reduction", mean)
    print("MC error with no variance reduction", MC_error)
    print("Price with antithetic", mean_anti)
    print("MC error with antithetic", MC_error_anti)

Here we price our certificate and use the antithetic variance reduction, as we can see we got an improvement. Our variance has been reduced approximately by 0.15...

In [17]:
MCS_antithetic(763.91, 1000, .02, 0.2, .03, .07, .2, 0.09, 0.4, 7, .5, 1000, 250)

Price with no variance reduction 1154.9160090042665
MC error with no variance reduction 0.963252145569879
Price with antithetic 1162.201506114045
MC error with antithetic 0.8114578072824989


The computation of the relevant Greeks are made using the centered finite difference, the only different formula is given by Gamma because it's the second derivative.

In [18]:
#Implementation for delta and gamma greek
delta_s = S0 * .01
mean = MCS(763.91, 1000, .02, 0.2, .03, .07, .2, 0.09, 0.4, 7, .5, 1000, 250)
s_up = S0 + delta_s
s_down = S0 - delta_s
price_d_up = MCS(s_up, 1000, .02, 0.2, .03, .07, .2, 0.09, 0.4, 7, .5, 1000, 250)
price_d_down = MCS(s_down, 1000, .02, 0.2, .03, .07, .2, 0.09, 0.4, 7, .5, 1000, 250)

#Implementation for theta
delta_t = 1
t_plus = maturity + 1
t_minus = maturity - 1
price_t_up = MCS(763.91, 1000, .02, 0.2, .03, .07, .2, 0.09, 0.4, t_plus, .5, 1000, 250)
price_t_down = MCS(763.91, 1000, .02, 0.2, .03, .07, .2, 0.09, 0.4, t_minus, .5, 1000, 250)

#Implementation for vega
delta_v = math.sqrt(v0) * .01
v_plus = (math.sqrt(v0) + delta_v)**2
v_minus = (math.sqrt(v0) - delta_v)**2
price_v_up = MCS(763.91, 1000, .02, 0.2, .03, .07, .2, 0.09, v_plus, 7, .5, 1000, 250)
price_v_down = MCS(763.91, 1000, .02, 0.2, .03, .07, .2, 0.09, v_minus, 7, .5, 1000, 250)

#Implementation for Rho
delta_r = r0 * .01
rho_plus = r0 + delta_r
rho_minus = r0 - delta_r
price_rho_up = MCS(763.91, 1000, rho_plus, 0.2, .03, .07, .2, 0.09, 0.4, 7, .5, 1000, 250)
price_rho_down = MCS(763.91, 1000, rho_minus, 0.2, .03, .07, .2, 0.09, 0.4, 7, .5, 1000, 250)

Delta = (price_d_up - price_d_down)/(2*delta_s)
Gamma = (price_d_up - 2*mean + price_d_down) / (delta_s**2)
Theta = -(price_t_up - price_t_down)/(2*delta_t)
Vega = (price_v_up - price_v_down) / (2*delta_v)
Rho = (price_rho_up - price_rho_down) / (delta_r*2)

print(Delta, Gamma, Theta, Vega, Rho)

0.9396959965226841 3.8963305320694565e-15 -41.84059003466007 93.37252564475173 -2708.061585844632


-The value of Delta suggests us that the price of our certificate has a high sensitivity to changes in underlying asset's price, which is true because the investment return (coupons and settlement amount) are strictly related to the performance of the underlying. For every unit of change we have a change in certificated price by 0.9397 in the same direction.
-The extremely low Gamma suggests us that the Delta is pretty much constant across a range of underlying asset price movements, as well has a really low sensitivity to changes in Delta.
-Here a negative Theta tells us that time to decay, meaning the increase in time to maturity by 1 year, decreases the value of our certificate. This can be realted to the cap of our certificate, imagine the index has a drift of 2% every year. Obviosuly with increasing time the chances that the underlying go over the cap become higher and higher reducing our certificate price.
-The increase of Vega by 1% has a positive impact other the certificate increasing the value by around 93$. This can be also true, because on the contrary of the stocks ,where 20% of standard deviation means increases and decreases of value by that percentage meaning they offset eachother, the options don't have a downside because we can choose not to exercise if the result gives us a loss. The certificate in our case has a barrier, so meaning it's similiar to options reality.
-The Rho is too high, means that we have computed the calculation in a wrong way or the numerical approximation needs a more complex model in order to get out the Rho.